In [0]:

#load adata silver 
df_silver = spark.read.parquet("/mnt/silverclientwind/wind_power_production_enriched")
#dime dates
df_silver.show(10)

+-------------+----------+--------+------------+--------+-------------+--------+---------+--------+--------------------+----------------------+----------+--------------+---------------+-----+---+-------+----+----+------+------+
|production_id|      date|    time|turbine_name|capacity|location_name|latitude|longitude|  region|              status|responsible_department|wind_speed|wind_direction|energy_produced|month|day|quarter|year|hour|minute|second|
+-------------+----------+--------+------------+--------+-------------+--------+---------+--------+--------------------+----------------------+----------+--------------+---------------+-----+---+-------+----+----+------+------+
|            1|2024-06-01|00-00-00|   Turbine A|    2200|   Location 1|    34.0|   -118.0|Region A|              Online|            Operations|  18.44936|            SE|     1786.91843|    6|  1|      2|2024|  00|    00|    00|
|            2|2024-06-01|00-00-00|   Turbine B|    2000|   Location 2|    37.0|   -119.

In [0]:


# 1. Créer le schéma Gold s'il n'existe pas
spark.sql("CREATE SCHEMA IF NOT EXISTS goldwind ")

#dim time 
dim_time=(df_silver
.select("time","hour","minute","second").distinct().withColumnRenamed("time","time_id")
)


# Écriture dans le layer Gold

# 4. Sauvegarder dans Gold
dim_time.write.format("delta").mode("overwrite").saveAsTable("goldwind.dim_time")


In [0]:
dim_date=(df_silver
.select("date","year","month","day","quarter").distinct()
.withColumnRenamed("date","date_id")

)

# 4. Sauvegarder dans Gold
dim_date.write.format("delta").mode("overwrite").saveAsTable("goldwind.dim_date")

In [0]:
#dim ostatus operation 
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
windowSpec = Window.orderBy("status", "responsible_department")

dim_status = (
    df_silver
    .select("status", "responsible_department")
    .distinct()
    .withColumn("status_id", row_number().over(windowSpec))
)

# 4. Sauvegarder dans Gold
dim_status.write.format("delta").mode("overwrite").saveAsTable("goldwind.dim_status")



In [0]:

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
windowSpec = Window.orderBy("turbine_name", "capacity","latitude","longitude","region")
dim_turbine = (
    df_silver
    .select("turbine_name", "capacity","latitude","longitude","region")
    .distinct()
    .withColumn("turbine_id", row_number().over(windowSpec))
)


# 4. Sauvegarder dans Gold
dim_turbine.write.format("delta").mode("overwrite").saveAsTable("goldwind.dim_turbine")


In [0]:
# Jointures avec les dimensions
fact = (
    df_silver
    # Join avec dim_date
    .join(dim_date, df_silver["date"] == dim_date["date_id"], "left")
    # Join avec dim_time
    .join(dim_time, df_silver["time"] == dim_time["time_id"], "left")
    # Join avec dim_turbine
    .join(dim_turbine, ["turbine_name", "capacity", "latitude", "longitude", "region"], "left")
    # Join avec dim_status
    .join(dim_status, ["status", "responsible_department"], "left")
    .select(
        "production_id",
        "date_id",
        "time_id",
        "turbine_id",
        "status_id",
        "wind_speed",
        "wind_direction",
        "energy_produced"
    )
)
# 4. Sauvegarder dans Gold
fact.write.format("delta").mode("overwrite").saveAsTable("goldwind.fact")


In [0]:
%sql
select * from goldwind.fact limit 10


production_id,date_id,time_id,turbine_id,status_id,wind_speed,wind_direction,energy_produced
1,2024-06-01,00-00-00,1,4,18.44936,SE,1786.91843
2,2024-06-01,00-00-00,2,4,16.34197,E,1156.38082
3,2024-06-01,00-00-00,3,4,24.16424,E,1216.49768
4,2024-06-01,00-10-00,1,4,13.03947,NE,1888.39496
5,2024-06-01,00-10-00,2,4,4.00397,SE,1052.36231
6,2024-06-01,00-10-00,3,6,5.22807,N,0.0
7,2024-06-01,00-20-00,1,6,20.38065,SE,0.0
8,2024-06-01,00-20-00,2,4,17.61423,NW,1459.88153
9,2024-06-01,00-20-00,3,4,23.72739,NW,658.86141
10,2024-06-01,00-30-00,1,4,6.28564,E,1184.2255
